In [1]:
import time
from collections import defaultdict
import pandas as pd
import numpy as np

## Read data

In [2]:
data = pd.read_csv('../data/raw_customer_visists_filtered.csv',
                   parse_dates=['date'])

data.head()

,tab_id,user_id,date,venue_chain_id,venue_id
0,000a45b6-ff23-4d42-95b7-732ee8887d2e,6a0c9cd3-edd7-08dd-ec10-fcf2def42345,2023-12-01,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
1,000c7df0-fed1-46e0-bc05-cf2b2b193c9f,CASH,2023-11-17,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
2,000d2405-9ee8-4cfb-ae2a-45200ef2c8f7,cc8f8d2e-4e62-a29f-ab53-10e268475a90,2023-12-21,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
3,000edf49-4595-4afd-8ae3-990e60ae6c57,cf8b1a82-f0f6-07bd-ad8b-2461ca248849,2023-11-14,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
4,001631be-c413-4aa0-b388-51170ce932bd,CASH,2023-12-23,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38


In [3]:
start_time = time.time()

## General Overview

The goal is to efficiently track user visits to venues on a monthly basis using a dictionary structure like {'user B': 3, 'user C': 4}, where values represent the number of visits within the current month. This approach ensures high performance by maintaining an incrementally updated dictionary that consolidates historical data for each venue.

So the main algorithm is as follows:
- each venue maintains a dictionary storing visit counts per user for the current month
- these dictionaries are merged across all previous months, continuously updating visit counts
- if users appear in the current month but were absent in historical data, they are added as a new visitors, otherwise, users' visit counters are updated
- the (shifted0 aggregated dictionary (per venue) reflects all past visits, enabling a fast comparison between current month's and historical data

Further this approac will be illustrated on some synthetic data.

## Data Preparation

This is to address
> If 1 customer has more than 1 bill/tabs within 1 business day - this is treated as 1 visit.

In [4]:
daily = (data
         .groupby([
             'venue_chain_id',
             'venue_id',
             'user_id',
             pd.Grouper(key='date', freq='D')])
         .size()
         .rename('num_visits')
         .reset_index()
         .assign(num_visits=1))

daily.head(15)

,venue_chain_id,venue_id,user_id,date,num_visits
0,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,00008bcb-88a6-fc9a-a9e3-919154cde457,2022-01-21,1
1,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0000dd41-e592-abf1-6bc7-3cf28cfb34a5,2023-08-12,1
2,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0001010b-7eb3-43dc-3108-494e9f864af9,2021-06-21,1
3,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0001010b-7eb3-43dc-3108-494e9f864af9,2021-09-09,1
4,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,000140eb-3ea3-cf2c-69b4-36860eb96e08,2022-12-14,1
5,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0001e9ad-5aa3-a435-7e99-3378cbc0c7cd,2023-07-25,1
6,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0001e9ad-5aa3-a435-7e99-3378cbc0c7cd,2023-08-24,1
7,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0002f699-1c9f-3e9b-d054-2360ca5f5adf,2021-10-05,1
8,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,0002fcbc-6e02-4f77-506a-9c8df49d8105,2022-09-09,1
9,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,00033c71-9b35-0be4-ca38-933959de73d9,2021-09-30,1


This is to address
> The system has a category Non-Returning customer. Non-Returning customer is a customer that hasn't visited a venue 365 days or more since the previous visit. It should be considered during calculation of new and returning customer statistics, meaning that if you see a visit of a specifi c customer at some data point and the next visit was in 2 years, it means that in 2 years this customer will be calculated again as ‘new’.

Thus, users who reappear after a long absence will be marked and further removed from the aggregated dictionary in the month they return.

The current approach for identifying non-returning users is a simplified approximation and may not be entirely precise. A more refined solution should be incorporated directly into the dictionary aggregation function to handle this logic properly. However, this implementation serves to demonstrate a fast and efficient method for tackling the problem.

In [5]:
daily['diff'] = (daily
                 .groupby([
                     'venue_chain_id',
                     'venue_id',
                     'user_id'])['date']
                 .diff().dt.days)

daily['is_non_return'] = np.where(
    (daily['user_id'] != 'CASH') & (daily['diff'] > 365), 1, 0)

daily[daily['user_id'] == '001077f2-8d85-859f-58b6-146929d4a948']

,venue_chain_id,venue_id,user_id,date,num_visits,diff,is_non_return
54,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,001077f2-8d85-859f-58b6-146929d4a948,2021-06-18,1,NaN,0
55,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,001077f2-8d85-859f-58b6-146929d4a948,2021-07-02,1,14.0,0
56,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,001077f2-8d85-859f-58b6-146929d4a948,2021-11-05,1,126.0,0
57,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,001077f2-8d85-859f-58b6-146929d4a948,2023-01-14,1,435.0,1


now this is how we will get dictionaries with user visits counts per month

In [6]:
monthly = (daily
           .groupby([
               'venue_chain_id',
               'venue_id',
               pd.Grouper(key='date', freq='ME'),
               'user_id'],
               sort=False)['num_visits']
           .sum()
           .reset_index())

monthly_counts = (monthly
                  .groupby([
                      'venue_chain_id',
                      'venue_id',
                      'date'])
                  .apply(lambda x:
                         dict(zip(x['user_id'], x['num_visits'])),
                         include_groups=False)
                  .rename('user_counts')
                  .to_frame())

monthly_counts.head(10)

user_counts
venue_chain_id                       venue_id date                                                         
14eaeec5-2fd0-621a-49aa-4a8090cabcf7 1        2021-06-30  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...
                                              2021-07-31  {'000ab454-dd24-b71a-2c47-84090a2b8477': 1, '0...
                                              2021-08-31  {'000ee134-02b4-925e-35ed-3a1e4ca6c3bf': 1, '0...
                                              2021-09-30  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...
                                              2021-10-31  {'0002f699-1c9f-3e9b-d054-2360ca5f5adf': 1, '0...
                                              2021-11-30  {'000d2d99-e40c-a64c-68a8-83407b587bf2': 1, '0...
                                              2021-12-31  {'0003fcf5-9c9f-2bac-cfd1-0f2b84c9b978': 1, '0...
                                              2022-01-31  {'00008bcb-88a6-fc9a-a9e3-919154cde457': 1, '0...
                                              2022-02-28  {'0008e9cc-4cf7-5063-52b9-e24caceb9b07': 1, '0...
                                              2022-03-31  {'001a6a81-d105-7ba6-cc1c-c458dd31b228': 1, '0...

And this is how we will aggragate the data to store historial (accumulated) data

In [7]:
def expand_counts(exp_series):
    res = defaultdict(int)
    for d in exp_series:
        for k, v in d.items():
            res[k] += v
    return dict(res)


monthly_counts['exp_counts'] = (
    monthly_counts
    .groupby([
        'venue_chain_id',
        'venue_id'
    ])['user_counts']
    .transform(lambda g:
               (g.apply(lambda x: [x])
                .cumsum()
                .apply(expand_counts))))

monthly_counts.head(10)

user_counts                                         exp_counts
venue_chain_id                       venue_id date                                                                                                            
14eaeec5-2fd0-621a-49aa-4a8090cabcf7 1        2021-06-30  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...
                                              2021-07-31  {'000ab454-dd24-b71a-2c47-84090a2b8477': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...
                                              2021-08-31  {'000ee134-02b4-925e-35ed-3a1e4ca6c3bf': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...
                                              2021-09-30  {'0001010b-7eb3-43dc-3108-494e9f864af9': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2021-10-31  {'0002f699-1c9f-3e9b-d054-2360ca5f5adf': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2021-11-30  {'000d2d99-e40c-a64c-68a8-83407b587bf2': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2021-12-31  {'0003fcf5-9c9f-2bac-cfd1-0f2b84c9b978': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2022-01-31  {'00008bcb-88a6-fc9a-a9e3-919154cde457': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2022-02-28  {'0008e9cc-4cf7-5063-52b9-e24caceb9b07': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...
                                              2022-03-31  {'001a6a81-d105-7ba6-cc1c-c458dd31b228': 1, '0...  {'0001010b-7eb3-43dc-3108-494e9f864af9': 2, '0...

This is an initial approach to addressing the problem of non-returning visitors, with room for further optimization and refinement. While it correctly identifies non-returning users as new within a given month, it may introduce inconsistencies when handling their subsequent visits. This method needs to be revised to ensure accuracy across multiple months. As of now we simply remove them from the aggregated dict which may be a suboptimal solution.

In [8]:
monthly_non_return = (daily
                      .groupby([
                          'venue_chain_id',
                          'venue_id',
                          pd.Grouper(key='date', freq='ME')],
                          sort=False)['user_id']
                      .apply(set)
                      .rename('non_return_set'))

monthly_counts = monthly_counts.join(monthly_non_return)

# to be further improved and optimised
monthly_counts['exp_counts'] = (
    monthly_counts
    .apply(lambda r:
           {k: v for k, v in r['exp_counts'].items()
            if k not in r['non_return_set']},
           axis=1))

Now, we shift the aggregated dictionary to enable a direct comparison between the current month’s visits and all historical data accumulated up to the last day of the previous month. This ensures that each month’s analysis is based on a clear separation between past and present data, allowing for an accurate identification of new vs. returning users.

In [9]:
monthly_counts['exp_counts'] = monthly_counts['exp_counts'].shift()

monthly_counts.dropna(inplace=True)

## Demo of the suggested approach

Basically, this is how the entire process will work, illustrated using a toy synthetic dataset.

In [10]:
demo = pd.DataFrame({
    'date': pd.to_datetime([
        '2024-01-31',
        '2024-02-29',
        '2024-03-31',
        '2024-04-30']),
    'user_counts': [
        {'A': 1, 'B': 1},
        {'B': 3, 'C': 4},
        {'A': 2, 'C': 1, 'D': 5},
        {'B': 2, 'D': 3, 'E': 4}
    ]
})

demo['exp_counts'] = (demo['user_counts']
                      .apply(lambda x: [x])
                      .cumsum()
                      .apply(expand_counts)
                      .shift())

demo.dropna(inplace=True)

demo[['new', 'return', 'repeat']] = (
    demo
    .apply(
        lambda r:
            pd.Series({
                'new': set(r['user_counts']) - set(r['exp_counts']),
                'return': set(r['user_counts']) & {k for k, v in r['exp_counts'].items() if v == 1},
                'repeat': set(r['user_counts']) & {k for k, v in r['exp_counts'].items() if v >= 2}
            }),
        axis=1
    ))

demo

,date,user_counts,exp_counts,new,return,repeat
1,2024-02-29,"{'B': 3, 'C': 4}","{'A': 1, 'B': 1}",{C},{B},{}
2,2024-03-31,"{'A': 2, 'C': 1, 'D': 5}","{'A': 1, 'B': 4, 'C': 4}",{D},{A},{C}
3,2024-04-30,"{'B': 2, 'D': 3, 'E': 4}","{'A': 3, 'B': 4, 'C': 5, 'D': 5}",{E},{},"{B, D}"


Here’s a more structured explanation of how the system works:

- user_counts – tracks the number of visits by users in the current month
- exp_counts – contains aggregated user visit counts up to the end of the previous month
- new* –identifies new users who are appearing for the first time in the current month
- return – identifies returning users who were present in previous months and are visiting the venue again
- repeat – represents users who have visited the venue multiple times in the current month

> If a customer pays with a credit card that appears for the very first time in the system, -

this is considered as the user appearing for the first time at the given venue, as suggested by the general context of the task

Now are the main calculations:

In [11]:
stats = (
    monthly_counts
    .apply(
        lambda r:
            pd.Series({
                'new': len(set(r['user_counts']) - set(r['exp_counts'])),
                'return': len(set(r['user_counts']) & {k for k, v in r['exp_counts'].items() if v == 1}),
                'return_id': set(r['user_counts']) & {k for k, v in r['exp_counts'].items() if v == 1},
                'repeat': len(set(r['user_counts']) & {k for k, v in r['exp_counts'].items() if v >= 2})
            }),
        axis=1
    ))

stats.head(10)

new  ...  repeat
venue_chain_id                       venue_id date              ...        
14eaeec5-2fd0-621a-49aa-4a8090cabcf7 1        2021-07-31  4908  ...       0
                                              2021-08-31  4038  ...      17
                                              2021-09-30  4190  ...      85
                                              2021-10-31  4607  ...     134
                                              2021-11-30  4238  ...     128
                                              2021-12-31  4455  ...     218
                                              2022-01-31  3832  ...     214
                                              2022-02-28  3974  ...     264
                                              2022-03-31  4467  ...     281
                                              2022-04-30  4343  ...     309

[10 rows x 4 columns]

## Customer conversion statistics
> For all ‘fi rst returned customers’ within a certain month, you need to calculate the time that is needed for this customer to be converted from ‘new’ to ‘fi rst returning’.

Now that we’ve identified all new users, we can merge their user IDs with the daily data, which includes the difference in days between visits. This allows to calculate the aggregated statistics.

In [12]:
conversion = (stats
              .explode('return_id')
              .reset_index()
              .drop(['new', 'return', 'repeat'],
                    axis=1)
              .pipe(lambda x:
                    x[x['return_id'] != 'CASH']))

conversion = (conversion
              .merge(
                  (daily[[
                      'venue_chain_id',
                      'venue_id',
                      'user_id',
                      'diff']]
                   .dropna()
                   .drop_duplicates(['venue_chain_id',
                                     'venue_id',
                                     'user_id'
                                     ])
                   ),
                  left_on=['venue_chain_id', 'venue_id', 'return_id'],
                  right_on=['venue_chain_id', 'venue_id', 'user_id']))

conversion['diff_bins'] = pd.cut(
    conversion['diff'], [-np.inf, 30, 60, 90, 180, 365, np.inf])

result = (conversion
          .groupby([
              'venue_chain_id',
              'venue_id',
              'date', 'diff_bins'],
              observed=False)
          .size()
          .unstack(fill_value=0))

result.head(15)

diff_bins                                                 (-inf, 30.0]  ...  (365.0, inf]
venue_chain_id                       venue_id date                      ...              
14eaeec5-2fd0-621a-49aa-4a8090cabcf7 1        2021-05-31             0  ...             0
                                              2021-06-30             0  ...             0
                                              2021-07-31             0  ...             0
                                              2021-08-31             0  ...             0
                                              2021-09-30             0  ...             0
                                              2021-10-31             0  ...             0
                                              2021-11-30             0  ...             0
                                              2021-12-31             0  ...             0
                                              2022-01-31             0  ...             0
                                              2022-02-28             0  ...             0
                                              2022-03-31             0  ...             0
                                              2022-04-30             0  ...             0
                                              2022-05-31             0  ...             0
                                              2022-06-30             0  ...             3
                                              2022-07-31             0  ...            23

[15 rows x 6 columns]

## Average monthly customers

In [13]:
stats['return_repeat'] = stats['return'] + stats['repeat']

stats[['new_12m_mean', 'return_12m_mean']] = (
    stats
    .groupby([
        'venue_chain_id',
        'venue_id'])[['new', 'return_repeat']]
    .apply(lambda g:
           g.rolling(
               window=12,
               min_periods=12)
           .mean())
    .reset_index(level=[0, 1], drop=True)
)

stats_out = (stats[[
    'new',
    'return',
    'repeat',
    'new_12m_mean',
    'return_12m_mean'
]]
    .dropna())

stats_out.head(15)

new  ...  return_12m_mean
venue_chain_id                       venue_id date              ...                 
14eaeec5-2fd0-621a-49aa-4a8090cabcf7 1        2022-06-30  4093  ...       531.166667
                                              2022-07-31  4347  ...       601.250000
                                              2022-08-31  4193  ...       659.666667
                                              2022-09-30  4254  ...       705.833333
                                              2022-10-31  4129  ...       745.916667
                                              2022-11-30  4079  ...       780.666667
                                              2022-12-31  5200  ...       824.750000
                                              2023-01-31  4225  ...       856.416667
                                              2023-02-28  4168  ...       878.166667
                                              2023-03-31  4203  ...       902.333333
                                              2023-04-30  3599  ...       910.833333
                                              2023-05-31  3740  ...       921.083333
                                              2023-06-30  3840  ...       924.166667
                                              2023-07-31  3582  ...       923.500000
                                              2023-08-31  3298  ...       921.416667

[15 rows x 5 columns]

In [14]:
end_time = time.time()
print(f'Elapsed time: {end_time - start_time:.2f} sec')

Elapsed time: 29.02 sec


## Save to csv

> Output results need to be saved in .csv format and aggregated on a monthly basis for single venues, as well as for the whole venue chain.

Although aggregation for the entire venue chain is not included, as it might be considered somewhat trivial, the format for this data outpuy is uncertain. Not sure whether it should be saved as a separate file or appended to the main file with a venue chain identifier

In [15]:
stats_out.to_csv('../derived/stats.csv')
result.to_csv('../derived/conversion.csv')